In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [2]:
# 二乗和誤差
def mean_squared_error(y,t):
    return 0.5*np.sum(np.square(y-t))

In [3]:
# 交差エントロピー誤差
def cross_entropy_error(y,t,eps=1e-7):
    return -np.sum(t*np.log(y+eps))

In [4]:
dataset = pd.read_csv("./train.csv")

In [5]:
# datasetを読み込む
t_temp = dataset[[0]].values.ravel().astype(np.uint8)
x_dataset = dataset.iloc[:,1:].values.astype(np.uint8)
n_dataset = len(t_temp)
t_dataset = np.zeros((n_dataset,10),np.uint8)
t_dataset[np.arange(n_dataset),t_temp]=1 # 1-hot 表現にする

# ランダムシャッフルする
index = np.arange(n_dataset)
np.random.shuffle(index)

x_dataset = x_dataset[index]
t_dataset = t_dataset[index]

# 教師データとテストデータに分割
n_test = n_dataset / 4
n_train = n_dataset - n_test 
x_train , t_train, x_test, t_test = \
    x_dataset[:n_train],t_dataset[:n_train],x_dataset[n_train:],t_dataset[n_train:]

In [6]:
def cross_entropy_error2(y,t,eps=1e-7):
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+eps))/batch_size

In [7]:
def numerical_diff(f,x,eps=1e-8):
    return (f(x+eps)-f(x-eps))/(2*eps)

In [8]:
def func(x):
    return 0.01*x**2+0.1*x

def f_diff(x):
    return 0.02*x+0.1



In [17]:
def numerical_gradient(f,x,eps=1e-8):
    f0 = f(x)
    grad = np.zeros_like(x)
    #print x
    for i in range(len(x)):
        x[i]+=eps
        f1 = f(x)
        x[i]-=eps
        grad[i] = (f1-f0)/eps
    
    return grad

numerical_gradient(lambda x:x[0]**2+x[1]**2,[1.0,1.0])


array([ 1.99999999,  1.99999999])

In [21]:
def gradient_descent(f,init_x,lr=0.01,step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr * grad
    
    return x

f = lambda x:x[0]**2+x[1]**2
print(gradient_descent(f,[1,1],lr=0.1))

[ -4.77083345e-09  -4.77083345e-09]


In [23]:
def softmax(x):
    """ common/functions.py  """
    
    
    # mnistとかやる場合は2次元配列(データ数x入力次元)
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x,axis=0)
        exp_x = np.exp(x)
        y = exp_x / np.sum(exp_x,axis=0)
        return y.T
    
    x = x - np.max(x)
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# print( softmax(np.array([0.0,0.1,0.2])))

[ 0.30060961  0.33222499  0.3671654 ]


In [86]:
def cross_entropy_error(y,t,eps=1e-8):
    """ common/functions.py  """
    
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
    
    # one-hot-vectorから正解ラベルのインデックスに
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    # print y
    # print y[np.arange(batch_size),t]
    return -np.sum(np.log(y[np.arange(batch_size),t]+eps))/batch_size

cross_entropy_error(np.array([[0.2,0.1,0]]),np.array([[0,0,1]]))

18.420680743952367

In [84]:
def numerical_gradient(f,x,eps=1e-8):
    """ common/gradient.py """
    f0 = f(x)
    grad = np.zeros_like(x)

    # http://www.aipacommander.com/entry/2017/05/14/172220    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        i = it.multi_index
        
        x[i]+=eps
        f1 = f(x)
        x[i]-=eps
        grad[i] = (f1-f0)/eps
        
        it.iternext()
    
    return grad

numerical_gradient(lambda x:x[0]**2+x[1]**2,np.array([1.0,1.0]))


array([ 1.99999999,  1.99999999])

In [92]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # ガウス分布で初期化
    
    def predict(self,x):
        return np.dot(x,self.W)
    
    def loss(self,x,t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y,t)
        
        return loss

net = simpleNet()
x = np.array([0.6,0.9])
p = net.predict(x)
t = np.array([0,0,1])

print(net.W)
print(p)
print(np.argmax(p))
print(net.loss(x,t))

[[ 0.58996193 -1.86691623  0.55723567]
 [-0.79245384 -0.84064432 -1.51343385]]
[-0.3592313  -1.87672963 -1.02774906]
0
1.21763709478


In [95]:
f = lambda W: net.loss(x,t)
dW = numerical_gradient(f,net.W)
print(dW)

[[ 0.34647487  0.07596797 -0.42244284]
 [ 0.51971227  0.11395196 -0.63366423]]
